In [76]:
import pandas as pd
import altair as alt
import numpy as np
alt.data_transformers.enable('default', max_rows=20000)
df = pd.read_pickle('../Final_Data/combined_data.pkl')
nest = df.set_index('series').dropna().to_dict()['Parent Series ID'] # Gives the series ID of the parent of a given series.
cat_names = df.set_index('series').dropna().to_dict()['Category'] # Gives the category name of a given series ID.

In [34]:
df

,year,period,periodName,value,footnotes,series,latest,Category,Parent Series ID,Level,Leaf,date,Type,Bucket
0,1989,M12,December,126.300000,[{}],CUSR0000SA0,NaN,CPI - All items,NaN,0.0,0.0,1989-12-01,NaN,NaN
1,1989,M11,November,125.900000,[{}],CUSR0000SA0,NaN,CPI - All items,NaN,0.0,0.0,1989-11-01,NaN,NaN
2,1989,M10,October,125.400000,[{}],CUSR0000SA0,NaN,CPI - All items,NaN,0.0,0.0,1989-10-01,NaN,NaN
3,1989,M09,September,124.800000,[{}],CUSR0000SA0,NaN,CPI - All items,NaN,0.0,0.0,1989-09-01,NaN,NaN
4,1989,M08,August,124.500000,[{}],CUSR0000SA0,NaN,CPI - All items,NaN,0.0,0.0,1989-08-01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1872,2022,NaN,NaN,4435.980526,NaN,NaN,NaN,S - S&P,NaN,NaN,NaN,2022-02-01,Stocks,NaN
1873,2022,NaN,NaN,4391.265217,NaN,NaN,NaN,S - S&P,NaN,NaN,NaN,2022-03-01,Stocks,NaN
1874,2022,NaN,NaN,4391.296000,NaN,NaN,NaN,S - S&P,NaN,NaN,NaN,2022-04-01,Stocks,NaN
1875,2022,NaN,NaN,4040.360000,NaN,NaN,NaN,S - S&P,NaN,NaN,NaN,2022-05-01,Stocks,NaN


In [35]:
def build_arg_text(**args):
    
    arg_text = '?chart_type=' + args.get('chart_type','Line+Chart')
    arg_text += '&start_year=' + args.get('start_year','2000')
    arg_text += '&end_year=' + args.get('end_year','2021')
    arg_text += '&inflation=' + args.get('inflation','By_Category')
    arg_text += '&earnings=' + args.get('earnings','Exclude')
    arg_text += '&unemployment=' + args.get('unemployment','Exclude')
    arg_text += '&stocks=' + args.get('stocks','Exclude')
    arg_text += '&interest=' + args.get('interest','Exclude')
    
    return arg_text

In [93]:
def data_parse(df, **args):
    
    website = 'https://apps-summer22.ischool.berkeley.edu/inflation_viz/chart'
    arg_text = website + build_arg_text(**args)
    
    start_date = '1/1/' + str(args.get('start_year','2000'))
    end_date = '12/1/' + str(args.get('end_year','2021'))
    start_year = int(args.get('start_year',2000))
    end_year = int(args.get('end_year',2021))
    
    # Start off with no hyperlinks and add where needed.
    df['href'] = arg_text
    
    # CPI
    df_cpi = pd.DataFrame()
    if args.get('parent','') == '':
        if args.get('inflation','By+Category') == 'Exclude':
            df_cpi = pd.DataFrame()
        elif args.get('inflation','By+Category') == 'By+Category':
            df_cpi = df[df['Parent Series ID'] == 'CUSR0000SA0']
        elif args.get('inflation','By+Category') == 'Total':
            df_cpi = df[df['series'] == 'CUSR0000SA0']
    else:
        df_cpi = df[df['Parent Series ID'] == args.get('parent','')]
        
    if args.get('inflation','By+Category') != 'Exclude' and len(df_cpi) > 0:
        df_cpi['href'] = np.where(df_cpi['Leaf'] == 0, arg_text + '&parent=' + df_cpi['series'], 'None')
        
    # Earnings
    if args.get('earnings','') in ['','Exclude']:
        df_earnings = pd.DataFrame()
    else:
        df_earnings = df[(df['Type'] == 'Earnings') & (df['Bucket'] == args.get('earnings','').replace('+',' '))]
        
    # Unemployment
    if args.get('unemployment','') in ['','Exclude']:
        df_unemployment = pd.DataFrame()
    else:
        df_unemployment = df[(df['Type'] == 'Unemployment') & (df['Bucket'] == args.get('unemployment','').replace('+',' '))]
        
    # Stocks
    if args.get('stocks','') in ['','Exclude']:
        df_stocks = pd.DataFrame()
    else:
        df_stocks = df[df['Type'] == 'Stocks'] 
        
    # Interest
    if args.get('interest','') in ['','Exclude']:
        df_interest = pd.DataFrame()
    else:
        df_interest = df[df['Type'] == 'Interest'] 

    # Combine selected data.
    df = pd.concat([df_cpi, df_earnings, df_unemployment, df_stocks, df_interest])
    
    # Skip remaining steps if df is empty.
    if len(df) == 0:
        return df
    
    # Normalize values to % change in the specific category from start of date window.
    baseline_dict, ac_dict = {}, {}
    series_start_dt = df[['date','value','Category']].groupby('Category').date.min().to_dict()
    #min_dt = df[df['year'] == start_year][['date','value','Category']].groupby('Category').date.min().to_dict()
    min_dt = df[df['date'] >= start_date][['date','value','Category']].groupby('Category').date.min().to_dict()
    #for k,v in min_dt.items():
    #    min_dt[k] = max(v,pd.to_datetime(start_date))
    for k,v in min_dt.items():
        baseline_dict[k] = df[(df['Category'] == k) & (df['date'] == v)].value.item()
    df['baseline'] = df['Category'].map(baseline_dict) 
    df['change'] = df['value']/df['baseline'] - 1
    df['baseline_year'] = df['Category'].map(series_start_dt).dt.year
    df['partial_data'] = np.where(df['baseline_year'] > int(args.get('start_year','2000')), "Since " + df['baseline_year'].astype(str), "")
    
    # Looking up anual change
    cat_list = df['Category'].to_list()
    date_list = df['date'].to_list()
    val_list = df['value'].to_list()
    val_lookup = dict(zip(zip(cat_list,date_list), val_list))
    df['past_key']  = list(zip(df['Category'], df['date'] - pd.offsets.DateOffset(years=1)))
    df['past_val'] = df['past_key'].map(val_lookup)
    df['yoy_change'] = df['value'] / df['past_val'] - 1
    
    return df

In [37]:
def build_line(df, **args):
    
    # Set dates.
    start_year = args.get('start_year',2000)
    start_date = '1/1/' + str(start_year)
    end_year = args.get('end_year',2021)
    end_date = '12/1/' + str(end_year)
    
    # Title
    if end_year != start_year:
        c_title = 'Change ' + str(start_year) + ' to ' + str(end_year)
    else:
        c_title = 'Change during ' + str(start_year)
    
    # Set color scheme based on the number of categories.
    if len(df['Category'].unique()) > 10:
        c_scheme = 'category20'
    else:
        c_scheme = 'category10'
        
    # Line Chart
    t_chart = alt.Chart(df[['date','change','Category','href']][(df['date'] >= start_date) & (df['date'] <= end_date) & (df['href'] != '')], title=c_title + ' by Category').mark_line(strokeWidth=2.5).encode(
            x = alt.X('date', title = 'Year'),
            y = alt.Y('change', title = c_title, axis=alt.Axis(format='%')),
            color = alt.Color('Category', scale=alt.Scale(scheme = c_scheme)),
            tooltip = 'Category',
            href = alt.Href('href')
        ).properties(height=400, width=600)
    
    t_chart['usermeta'] = {"embedOptions": {'loader': {'target': '_chart'}}}
    
    return t_chart

In [38]:
def build_bar(df, **args):
    
    # Set dates.
    start_year = args.get('start_year',2000)
    start_date = '1/1/' + str(start_year)
    end_year = args.get('end_year',2021)
    end_date = '12/1/' + str(end_year)
    
    # Create title.
    if end_year != start_year:
        c_title = 'Change ' + str(start_year) + ' to ' + str(end_year)
    else:
        c_title = 'Change during ' + str(start_year)
    
    # Set color scheme based on the number of categories.
    if len(df['Category'].unique()) > 10:
        c_scheme = 'category20'
    else:
        c_scheme = 'category10'
        
    # Bar Chart
    # ,title=c_title + ' by Category'
    t_chart = alt.Chart(df[['date','change','Category','href','partial_data']][(df['date'] == end_date) | ((df['periodName'] == '4th Quarter') & (df['year'] == end_year))]).mark_bar().encode(
            x = alt.X('Category', sort='y'),
            y = alt.Y('change', title=c_title, axis=alt.Axis(format='%')),
            color = alt.Color('Category', scale=alt.Scale(scheme = c_scheme)),
            tooltip = [alt.Tooltip('date:T', title = 'Date', format='%B %Y'), 'Category',
                   alt.Tooltip('change:Q',title = "Total Change",  format='.1%'), 
                   alt.Tooltip('partial_data',title = "Notes")],
            href = alt.Href('href')
        ).properties(height=400, width=600)

    
    t_chart['usermeta'] = {"embedOptions": {'loader': {'target': '_chart'}}}
    
    return t_chart

In [39]:
def build_line_v2(df, **args):
    
    # Set dates.
    start_year = args.get('start_year',2000)
    start_date = '1/1/' + str(start_year)
    ar_start_year = '1/1/' + str(int(start_year) - 1)
    end_year = args.get('end_year',2021)
    end_date = '12/1/' + str(end_year)
    
    # Create title.
    if end_year != start_year:
        c_title = 'Change ' + str(start_year) + ' to ' + str(end_year)
    else:
        c_title = 'Change during ' + str(start_year)

    df_test = df.loc[(df['date'] >= ar_start_year) & (df['date'] <= end_date)]
    
    # new data frame with only graph values
    graph_data = df_test[['date', 'Category', 'change','yoy_change', 'href']]
    graph_data['change'] = graph_data['change'].round(decimals = 3)
    graph_data['yoy_change'] = graph_data['yoy_change'].round(decimals = 3)
    graph_data = graph_data[graph_data['date'] >= start_date]
    
    s2 = pd.DataFrame(graph_data.Category.unique(), columns = ['Category'])
    highlight = alt.selection_multi(on = 'mouseover', fields=['Category'], nearest = True)

    line = alt.Chart(graph_data).mark_line(interpolate = 'basis').encode(
        x = alt.X('date:T', title = "Year"),
        y = alt.Y('change:Q', title = c_title, axis=alt.Axis(ticks = False, domain = False, format='%')),
        tooltip = [alt.Tooltip('date:T', title = 'Date', format='%B %Y'), 'Category',
                   alt.Tooltip('change:Q',title = "Cumulative Change",  format='.1%'), 
                   alt.Tooltip('yoy_change:Q',title = "Annual Change",  format='.1%')],    
        color = alt.Color('Category:N'),
        href = alt.Href('href'),
        opacity = alt.condition(highlight, alt.value(1), alt.value(0.2))).properties(
        width=600, height=400
    )

    hover_legend = alt.Chart(s2).mark_circle(size = 100).encode(
        y = alt.Y('Category:N', axis = alt.Axis(orient = 'right', domain = False, ticks = False), title = None),
        color = alt.Color('Category:N', legend = None),
        opacity = alt.condition(highlight, alt.value(1), alt.value(0.2))
    ).add_selection(
        highlight
    )
    
    t_chart = (line | hover_legend).configure_axis(
        grid=False).configure_view(
        strokeWidth = 0)
    
    
    t_chart['usermeta'] = {"embedOptions": {'loader': {'target': '_chart'}}}
    
    return t_chart



In [97]:
# Simulate args.
#arg_str = "chart_type=Line+Chart&start_year=2005&end_year=2015"
arg_str = "chart_type=Bar+Chart&start_year=1970&end_year=2021&earnings=Total"
#arg_str = "chart_type=Bar+Chart&start_year=1970&end_year=2021"
#arg_str = "chart_type=Line+Chart&start_year=1970&end_year=2021"
#arg_str = "chart_type=Line+Chart&start_year=2010&end_year=2021&inflation=By+Category&earnings=By+Race+and+Gender&unemployment=Include&stocks=Exclude&interest=Include"
#arg_str = "chart_type=Bar+Chart&start_year=1970&end_year=2021&inflation=Total&earnings=Total&unemployment=Exclude&stocks=Exclude&interest=Include"
#arg_str = "chart_type=Bar+Chart&start_year=1973&end_year=2021&inflation=Total&earnings=Exclude&unemployment=By+Education&stocks=Exclude"
#arg_str = "chart_type=Bar+Chart&start_year=1973&end_year=2021&inflation=By+Category&earnings=Exclude&unemployment=By+Education&stocks=Exclude"
#arg_str = "chart_type=Bar+Chart&start_year=2005&end_year=2021&inflation=By_Category&earnings=Exclude&unemployment=Exclude&stocks=Exclude"

arg_list = arg_str.split('&')
arg_dict = {z.split('=')[0] : z.split('=')[1] for z in arg_list}
args = arg_dict

t_df = data_parse(df.copy(deep=True), **args)

if len(t_df) > 0:
    if args['chart_type'] == 'Line+Chart':
        #out_html = build_line(t_df, **args)
        out_html = build_line_v2(t_df, **args)
    elif args['chart_type'] == 'Bar+Chart':
        out_html = build_bar(t_df, **args)
    else:
        out_html = ""
else:
    out_html = ""

out_html

<ipython-input-93-ce79cad6f98a>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cpi['href'] = np.where(df_cpi['Leaf'] == 0, arg_text + '&parent=' + df_cpi['series'], 'None')


alt.Chart(...)